In [58]:
import os

project_root = "/Users/nimisha/Desktop/learning/news-reliability-detector/"
os.chdir(project_root)

In [59]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, udf, concat
from pyspark.sql.types import StringType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

# local modules
from src.load.load_data import load
from src.preprocessing.preprocessing import (
    print_null_rows,
    drop_missing_text_rows,
    shuffle_rows,
    remove_stop_words,
    stem_words
)

In [60]:
spark = SparkSession.builder.appName("NewsReliability").getOrCreate()

In [61]:
data = load(spark)

In [62]:
data.show(10)

+--------------------+--------------------+-------+-----------------+------+
|               title|                text|subject|             date|target|
+--------------------+--------------------+-------+-----------------+------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|     1|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|     1|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|     1|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|     1|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|     1|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|     1|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|     1|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|     1|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|     1|
| WATCH: Brand-New...|Just when you mig...|   News|December 21, 2017|     1|

In [63]:
data.printSchema()

root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- target: integer (nullable = false)



# Handling Missing Values

In [64]:
print_null_rows(data)

+-----+----+-------+----+------+
|title|text|subject|date|target|
+-----+----+-------+----+------+
|    0|   8|      8|   8|     0|
+-----+----+-------+----+------+



In [65]:
data.filter(data.text.isNull()).show()

+--------------------+----+-------+----+------+
|               title|text|subject|date|target|
+--------------------+----+-------+----+------+
|                   "|NULL|   NULL|NULL|     1|
|Ex-GOP Congressma...|NULL|   NULL|NULL|     1|
|                   "|NULL|   NULL|NULL|     1|
|WATCH: Democratic...|NULL|   NULL|NULL|     1|
|                   "|NULL|   NULL|NULL|     1|
|Trump Gets STOMPE...|NULL|   NULL|NULL|     1|
|                   "|NULL|   NULL|NULL|     1|
|Donald Trump Gets...|NULL|   NULL|NULL|     1|
+--------------------+----+-------+----+------+



In [66]:
data = drop_missing_text_rows(data)

In [67]:
data.filter(data.text.isNull()).show()

+-----+----+-------+----+------+
|title|text|subject|date|target|
+-----+----+-------+----+------+
+-----+----+-------+----+------+



In [68]:
print_null_rows(data)

+-----+----+-------+----+------+
|title|text|subject|date|target|
+-----+----+-------+----+------+
|    0|   0|      0|   0|     0|
+-----+----+-------+----+------+



# Randomize the order of data rows

In [69]:
data = shuffle_rows(data)
data.show(10)

+--------------------+--------------------+------------+-------------------+------+
|               title|                text|     subject|               date|target|
+--------------------+--------------------+------------+-------------------+------+
|Philosopher Slavo...|21st Century Wire...|     US_News|  November 17, 2016|     1|
|Skeptical of Russ...|WASHINGTON (Reute...|politicsNews|September 20, 2016 |     0|
|BOILER ROOM – EP ...|Tune in to the Al...|     US_News|     March 24, 2016|     1|
|KEY TRUMP ADVISOR...|Elton John will p...|   left-news|       Nov 23, 2016|     1|
| Street Performer...|A street performe...|        News|     March 20, 2016|     1|
|PRO ABORTION PAC,...|Of course Emily s...|   left-news|       Apr 13, 2015|     1|
|Approval of Presi...|NEW YORK (Reuters...|politicsNews|      May 19, 2017 |     0|
|BOOM! MEXICAN AME...| Let me tell you ...|    politics|       Jun 29, 2016|     1|
|Senator McCain wi...|WASHINGTON (Reute...|politicsNews| December 14, 2017 |

## Feature Creating

In [70]:
# combine title and text column to make news content
data = data.withColumn('content', concat(data['title'], data['text']))
data.show(3)

+--------------------+--------------------+------------+-------------------+------+--------------------+
|               title|                text|     subject|               date|target|             content|
+--------------------+--------------------+------------+-------------------+------+--------------------+
|Philosopher Slavo...|21st Century Wire...|     US_News|  November 17, 2016|     1|Philosopher Slavo...|
|Skeptical of Russ...|WASHINGTON (Reute...|politicsNews|September 20, 2016 |     0|Skeptical of Russ...|
|BOILER ROOM – EP ...|Tune in to the Al...|     US_News|     March 24, 2016|     1|BOILER ROOM – EP ...|
+--------------------+--------------------+------------+-------------------+------+--------------------+
only showing top 3 rows



In [71]:
data.select("content").show(n = 5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Drop columns

In [72]:
columns_to_drop = ["date", "title", "subject", "text"]
data = data.drop(*columns_to_drop)
data.show(2)

+------+--------------------+
|target|             content|
+------+--------------------+
|     1|Philosopher Slavo...|
|     0|Skeptical of Russ...|
+------+--------------------+
only showing top 2 rows



# Duplicate Rows

In [73]:
duplicate_rows = data.groupBy(data.columns).count().filter("count > 1")

print("Duplicates", duplicate_rows.count())
print("Total", data.count())
print("Distinct", data.distinct().count())

Duplicates 5305
Total 44899


Distinct 39106


In [74]:
duplicate_rows.show(10)

+------+--------------------+-----+
|target|             content|count|
+------+--------------------+-----+
|     1|JUST IN: SUSPECTE...|    2|
|     1|YOU’LL NEVER GUES...|    2|
|     1|HOUSE INTEL Slaps...|    2|
|     1|FLASHBACK: CNN HO...|    2|
|     1|PRICELESS! WATCH ...|    2|
|     1|CNN’S FAKE NEWS B...|    2|
|     1|WATCH: MEGHAN MCC...|    2|
|     1|“MAXINE WATERS IN...|    2|
|     1|HEY CNN…REMEMBER ...|    2|
|     1|HOW PRESIDENT TRU...|    2|
+------+--------------------+-----+
only showing top 10 rows



In [75]:
data = data.dropDuplicates()
data.count()

39106

# Text Transform 

### 1. Lower Case

In [76]:
# convert to all lower case
data = data.withColumn("content", lower(col("content")))
data.show(5)

+------+--------------------+
|target|             content|
+------+--------------------+
|     1| trump bucks gop ...|
|     1| australia prime ...|
|     1| mccain f*cked ov...|
|     1| american psychoa...|
|     1| senate gives tru...|
+------+--------------------+
only showing top 5 rows



### 2. Remove Special Characters 
- like ! @

In [77]:
data = data.withColumn("content", regexp_replace("content", "[^a-zA-Z0-9\\s]", ""))
data.show(5)

+------+--------------------+
|target|             content|
+------+--------------------+
|     1| trump bucks gop ...|
|     1| australia prime ...|
|     1| mccain fcked ove...|
|     1| american psychoa...|
|     1| senate gives tru...|
+------+--------------------+
only showing top 5 rows



### 3. Remove Stopwords

In [78]:
remove_stop_words_udf = udf(remove_stop_words, StringType())
data = data.withColumn("content", remove_stop_words_udf("content"))
data.show(10)

+------+--------------------+
|target|             content|
+------+--------------------+
|     1|ca cops brutally ...|
|     1|trump monstrous t...|
|     1|maxine waters gli...|
|     0|trumps orders tar...|
|     1|transcript video ...|
|     1|trumps vp pick at...|
|     0|tech companies jo...|
|     1|breaking baltimor...|
|     0|juncker calls eur...|
|     1|wow trumps pick l...|
+------+--------------------+
only showing top 10 rows



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nimisha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 4. Stemming words

In [79]:
stem_words_udf = udf(stem_words, StringType())
data = data.withColumn("content", stem_words_udf("content"))
data.show(5)

[nltk_data] Downloading package stopwords to                        (0 + 1) / 1]
[nltk_data]     /Users/nimisha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


+------+--------------------+
|target|             content|
+------+--------------------+
|     1|ca cop brutal bea...|
|     1|trump monstrou th...|
|     1|maxin water glitt...|
|     0|trump order targe...|
|     1|transcript video ...|
+------+--------------------+
only showing top 5 rows



## Train Test Split

In [80]:
train_data, test_data = data.randomSplit(weights=[0.8, 0.2], seed=100)

In [81]:
# print(train_data.count())
# print(test_data.count())

In [82]:
feature_columns = ["content"]
target_column = "target"

X_train = train_data.select(*feature_columns)
X_test = test_data.select(*feature_columns)

y_train = train_data.select(target_column)
y_test = test_data.select(target_column)

In [83]:
# print(X_train.count())
# print(y_train.count())

# print(X_test.count())
# print(y_test.count())